In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime, timedelta
from scrapping import *
from fonctions_preprocess import * 
from script_automatisation import *

In [2]:
update_data()

/home/onyxia/work/Projet-python/Fbref_alex/scrapping.py:92: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  team_data = pd.read_html(team_response.text, match="Scores")[0]
/home/onyxia/work/Projet-python/Fbref_alex/scrapping.py:158: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/home/onyxia/work/Projet-python/Fbref_alex/scrapping.py:158: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/home/onyxia/work/Projet-python/Fbref_alex/scrapping.py:158: FutureWarning: Passing literal html to 'read_html' is deprecated and wil

In [5]:
a = pd.read_csv("/home/onyxia/work/Projet-python/Fbref_alex/SOCCER_241223.csv", index_col=0)

In [6]:
a = renommer_colonnes(a)

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Passes into Penalty Area,Crosses into Penalty Area,Progressive Passes,Shots on Target Against,Keeper Saves,Keeper Save Percentage,Clean Sheets,Penalty Kicks Against,Penalty Kicks Against Saved,Penalty Kicks Against Missed
0,2018-07-26,17:45,Europa Lg,Second qualifying round,Thu,Away,W,1,0,lv FK Ventspils,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0,0.0,0.0
1,2018-08-02,20:45,Europa Lg,Second qualifying round,Thu,Home,W,2,1,lv FK Ventspils,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0.0,0.0
2,2018-08-04,20:00,Trophée des Champions,Trophée des Champions,Sat,Neutral,L,0,4,Paris S-G,...,NaN,NaN,NaN,7.0,3.0,42.9,0.0,0,0.0,0.0
3,2018-08-04,20:00,Trophée des Champions,Trophée des Champions,Sat,Neutral,W,4,0,Monaco,...,NaN,NaN,NaN,2.0,2.0,100.0,1.0,0,0.0,0.0
4,2018-08-09,20:00,Europa Lg,Third qualifying round,Thu,Away,W,3,1,ua FC Mariupol,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4550,2023-12-20,21:00,Ligue 1,Matchweek 17,Wed,Home,W,3.0,1.0,Metz,...,8.0,2.0,72.0,1.0,0.0,0.0,0.0,0,0.0,0.0
4551,2023-12-20,21:00,Ligue 1,Matchweek 17,Wed,Home,W,1.0,0.0,Le Havre,...,15.0,2.0,85.0,1.0,1.0,100.0,1.0,0,0.0,0.0
4552,2023-12-20,21:00,Ligue 1,Matchweek 17,Wed,Away,W,3.0,1.0,Clermont Foot,...,14.0,6.0,64.0,6.0,5.0,83.3,0.0,0,0.0,0.0
4553,2023-12-20,21:00,Ligue 1,Matchweek 17,Wed,Home,W,2.0,1.0,Lille,...,9.0,2.0,28.0,2.0,2.0,50.0,0.0,0,0.0,0.0


In [22]:

def rate_limit():
    """
    Fonction pour limiter le taux de requêtes et éviter de surcharger le serveur.
    Utilise un délai d'attente entre les requêtes pour respecter les limitations.
    """
    MIN_REQUEST_INTERVAL = 2.5
    last_request_time = getattr(rate_limit, "last_request_time", None)
    if last_request_time is not None:
        elapsed_time = time.time() - last_request_time
        if elapsed_time < MIN_REQUEST_INTERVAL:
            time.sleep(MIN_REQUEST_INTERVAL - elapsed_time)
    rate_limit.last_request_time = time.time()


In [21]:
A = scrape_latest_ligue1_data()

/home/onyxia/work/Projet-python/Fbref_alex/scrapping.py:92: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  team_data = pd.read_html(team_response.text, match="Scores")[0]
/home/onyxia/work/Projet-python/Fbref_alex/scrapping.py:158: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/home/onyxia/work/Projet-python/Fbref_alex/scrapping.py:158: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/home/onyxia/work/Projet-python/Fbref_alex/scrapping.py:158: FutureWarning: Passing literal html to 'read_html' is deprecated and wil

In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier


mapping_equipe = {
    'Nimes': 'Nîmes',
    'Paris S-G': 'Paris Saint Germain',
    'Saint Etienne': 'Saint-Étienne'
}

a = pd.read_csv("recup_matchweek.csv", index_col=0)
b = pd.read_csv("SOCCER_241223.csv", index_col=0)
c = preparation_modelisation(base_initial=b, matchweeks=a, mapping_equipe= mapping_equipe)
c = preparation_model(c)




# Créer des variables factices pour la colonne 'Opponent'
dummies = pd.get_dummies(c['Opponent'])

# Fusionner les variables factices avec le DataFrame original
c = pd.concat([c, dummies], axis=1)



# Séparer les données en ensembles d'entraînement et de test
train_c = c[c['Result'].notna()]
test_c = c[c['Result'].isna()]


# Filtrer pour conserver uniquement les colonnes avec "_Lag1" et les colonnes des variables factices
features = [col for col in train_c.columns if '_Lag1' in col] + list(dummies.columns)


# Préparer les ensembles d'entraînement et de test
X_train = train_c[features]
y_train = train_c['Result']
X_test = test_c[features]

# Supprimer les lignes avec des NaN dans les colonnes de features dans l'ensemble d'entraînement
X_train = X_train.dropna()
y_train = y_train[X_train.index]  # Assurez-vous que y_train a les mêmes lignes que X_train

# Créer et entraîner le modèle Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

predictions = rf_model.predict(X_test)

# Ajouter les prédictions à l'ensemble de test
test_c['Predicted_Result'] = predictions


test_c = affichage_colonne_stockage(test_c)

test_c.to_csv("Resultats.csv")




In [7]:

pd.set_option('display.max_columns', None)
c.sort_values(by="DateTime", ascending=False)



,DateTime,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Saison,MatchID,Standard_Gls,Standard_Sh,Standard_SoT,Standard_SoT%,Standard_G/Sh,Standard_G/SoT,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_xG,Expected_npxG,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Take-Ons_Att,Take-Ons_Succ,Take-Ons_Succ%,Take-Ons_Tkld,Take-Ons_Tkld%,Carries_Carries,Carries_TotDist,Carries_PrgDist,Carries_PrgC,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Rec,Receiving_PrgR,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Challenges_Tkl,Challenges_Att,Challenges_Tkl%,Challenges_Lost,Blocks_Blocks,Blocks_Sh,Blocks_Pass,Int,Tkl+Int,Clr,Err,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,Ast,xAG,xA,KP,1/3,PPA,CrsPA,PrgP,Performance_SoTA,Performance_GA,Performance_Saves,Performance_Save%,Performance_CS,Performance_PSxG,Performance_PSxG+/-,Penalty Kicks_PKatt,Penalty Kicks_PKA,Penalty Kicks_PKsv,Penalty Kicks_PKm,Launched_Cmp,Launched_Att,Launched_Cmp%,Passes_Att (GK),Passes_Thr,Passes_Launch%,Passes_AvgLen,Goal Kicks_Att,Goal Kicks_Launch%,Goal Kicks_AvgLen,Crosses_Opp,Crosses_Stp,Crosses_Stp%,Sweeper_#OPA,Sweeper_AvgDist,GD,Points,Points_Cum,GD_Cum,GF_Cum,GA_Cum,Classement,Past_Matches,IsWin,IsDraw,IsLoss,CumulativeWins,CumulativeDraws,CumulativeLosses,Points_Cum_Lag1,GD_Cum_Lag1,GF_Cum_Lag1,GA_Cum_Lag1,Classement_Lag1,CumulativeWins_Lag1,CumulativeDraws_Lag1,CumulativeLosses_Lag1
3669,2024-01-14 20:45:00,Ligue 1,18,Sun,Home,NaN,NaN,NaN,Paris Saint Germain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Lens,2023-2024,Lens_Paris Saint Germain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0,4.0,21.0,17.0,7.0,2.0,2.0,3.0
3849,2024-01-14 20:45:00,Ligue 1,18,Sun,Away,NaN,NaN,NaN,Lens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Paris Saint Germain,2023-2024,Paris Saint Germain_Lens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,28.0,42.0,14.0,1.0,3.0,2.0,2.0
3723,2024-01-14 17:05:00,Ligue 1,18,Sun,Away,NaN,NaN,NaN,Le Havre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Lyon,2023-2024,Lyon_Le Havre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,-11.0,16.0,27.0,14.0,0.0,1.0,0.0
3651,2024-01-14 17:05:00,Ligue 1,18,Sun,Home,NaN,NaN,NaN,Lyon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Le Havre,2023-2024,Le Havre_Lyon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [10]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier




# Créer des variables factices pour la colonne 'Opponent'
dummies = pd.get_dummies(c['Opponent'])

# Fusionner les variables factices avec le DataFrame original
c = pd.concat([c, dummies], axis=1)



# Séparer les données en ensembles d'entraînement et de test
train_c = c[c['Result'].notna()]
test_c = c[c['Result'].isna()]


# Filtrer pour conserver uniquement les colonnes avec "_Lag1" et les colonnes des variables factices
features = [col for col in train_c.columns if '_Lag1' in col] + list(dummies.columns)


# Préparer les ensembles d'entraînement et de test
X_train = train_c[features]
y_train = train_c['Result']
X_test = test_c[features]

# Supprimer les lignes avec des NaN dans les colonnes de features dans l'ensemble d'entraînement
X_train = X_train.dropna()
y_train = y_train[X_train.index]  # Assurez-vous que y_train a les mêmes lignes que X_train

# Créer et entraîner le modèle Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

predictions = rf_model.predict(X_test)

# Ajouter les prédictions à l'ensemble de test
test_c['Predicted_Result'] = predictions




/tmp/ipykernel_554767/576423102.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_c['Predicted_Result'] = predictions


In [11]:
affichage_colonne(test_c)



,DateTime,Comp,Round,Day,Venue,Team,Classement_Lag1,Predicted_Result,Opponent,Result,GF,GA,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Saison,MatchID,Standard_Gls,Standard_Sh,Standard_SoT,Standard_SoT%,Standard_G/Sh,Standard_G/SoT,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_xG,Expected_npxG,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Take-Ons_Att,Take-Ons_Succ,Take-Ons_Succ%,Take-Ons_Tkld,Take-Ons_Tkld%,Carries_Carries,Carries_TotDist,Carries_PrgDist,Carries_PrgC,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Rec,Receiving_PrgR,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Challenges_Tkl,Challenges_Att,Challenges_Tkl%,Challenges_Lost,Blocks_Blocks,Blocks_Sh,Blocks_Pass,Int,Tkl+Int,Clr,Err,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,Ast,xAG,xA,KP,1/3,PPA,CrsPA,PrgP,Performance_SoTA,Performance_GA,Performance_Saves,Performance_Save%,Performance_CS,Performance_PSxG,Performance_PSxG+/-,Penalty Kicks_PKatt,Penalty Kicks_PKA,Penalty Kicks_PKsv,Penalty Kicks_PKm,Launched_Cmp,Launched_Att,Launched_Cmp%,Passes_Att (GK),Passes_Thr,Passes_Launch%,Passes_AvgLen,Goal Kicks_Att,Goal Kicks_Launch%,Goal Kicks_AvgLen,Crosses_Opp,Crosses_Stp,Crosses_Stp%,Sweeper_#OPA,Sweeper_AvgDist,GD,Points,Points_Cum,GD_Cum,GF_Cum,GA_Cum,Classement,Past_Matches,IsWin,IsDraw,IsLoss,CumulativeWins,CumulativeDraws,CumulativeLosses,Points_Cum_Lag1,GD_Cum_Lag1,GF_Cum_Lag1,GA_Cum_Lag1,CumulativeWins_Lag1,CumulativeDraws_Lag1,CumulativeLosses_Lag1,Ajaccio,Amiens,Angers,Auxerre,Bordeaux,Brest,Caen,Clermont Foot,Dijon,Guingamp,Le Havre,Lens,Lille,Lorient,Lyon,Marseille,Metz,Monaco,Montpellier,Nantes,Nice,Nîmes,Paris Saint Germain,Reims,Rennes,Saint-Étienne,Strasbourg,Toulouse,Troyes
3615,2024-01-14 15:00:00,Ligue 1,18,Sun,Home,Brest,4.0,W,Montpellier,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,2023-2024,Brest_Montpellier,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0,10.0,25.0,15.0,2.0,2.0,4.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
3633,2024-01-14 15:00:00,Ligue 1,18,Sun,Away,Clermont Foot,18.0,L,Nantes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,2023-2024,Clermont Foot_Nantes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,-15.0,11.0,26.0,0.0,2.0,3.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
3651,2024-01-14 17:05:00,Ligue 1,18,Sun,Home,Le Havre,12.0,L,Lyon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,2023-2024,Le Havre_Lyon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

Points_Cum_Lag1          0
GD_Cum_Lag1              0
GF_Cum_Lag1              0
GA_Cum_Lag1              0
Classement_Lag1          0
CumulativeWins_Lag1      0
CumulativeDraws_Lag1     0
CumulativeLosses_Lag1    0
dtype: int64

In [6]:


a = find_futur_matchweeks(a, mapping_equipe)

b = preprocess_intitial(b, mapping_equipe)

b = preprocess_variables(b)
b.head(1)

resultat = pd.concat([a, b], sort=False)

In [7]:
a.head(1)

,DateTime,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Saison
218,2024-01-12 21:00:00,Ligue 1,18,Fri,Home,NaN,NaN,NaN,Strasbourg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Marseille,2023-2024


In [11]:
pd.set_option('display.max_columns', None)

In [9]:

b = preprocess_variables(b)
b.head(1)

,DateTime,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Standard_Gls,Standard_Sh,Standard_SoT,Standard_SoT%,Standard_G/Sh,Standard_G/SoT,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_xG,Expected_npxG,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Take-Ons_Att,Take-Ons_Succ,Take-Ons_Succ%,Take-Ons_Tkld,Take-Ons_Tkld%,Carries_Carries,Carries_TotDist,Carries_PrgDist,Carries_PrgC,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Rec,Receiving_PrgR,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Challenges_Tkl,Challenges_Att,Challenges_Tkl%,Challenges_Lost,Blocks_Blocks,Blocks_Sh,Blocks_Pass,Int,Tkl+Int,Clr,Err,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,Ast,xAG,xA,KP,1/3,PPA,CrsPA,PrgP,Performance_SoTA,Performance_GA,Performance_Saves,Performance_Save%,Performance_CS,Performance_PSxG,Performance_PSxG+/-,Penalty Kicks_PKatt,Penalty Kicks_PKA,Penalty Kicks_PKsv,Penalty Kicks_PKm,Launched_Cmp,Launched_Att,Launched_Cmp%,Passes_Att (GK),Passes_Thr,Passes_Launch%,Passes_AvgLen,Goal Kicks_Att,Goal Kicks_Launch%,Goal Kicks_AvgLen,Crosses_Opp,Crosses_Stp,Crosses_Stp%,Sweeper_#OPA,Sweeper_AvgDist,Saison,GD,Points,Points_Cum,GD_Cum,GF_Cum,GA_Cum,Classement,MatchID,Past_Matches,IsWin,IsDraw,IsLoss,CumulativeWins,CumulativeDraws,CumulativeLosses
14,2018-08-12 15:00:00,Ligue 1,1,Sun,Away,L,0,2,Lyon,0.4,1.4,33.0,48263.0,Régis Gurtner,4-3-3,Franck Schneider,Match Report,NaN,Amiens,0,10.0,1.0,10.0,0.0,0.0,25.1,1.0,0,0,0.4,0.4,0.04,-0.4,-0.4,414.0,49.0,152.0,195.0,73.0,6.0,414.0,13.0,8.0,61.5,5.0,38.5,198.0,1296.0,710.0,14.0,10.0,2.0,14.0,8.0,211.0,25.0,23.0,8.0,14.0,7.0,2.0,12.0,28.0,42.9,16.0,17.0,5.0,12.0,14.0,37,17.0,0.0,216.0,301.0,71.8,3869.0,1596.0,101.0,125.0,80.8,84.0,100.0,84.0,24.0,61.0,39.3,0.0,0.3,0.1,7.0,18.0,2.0,0.0,26.0,7.0,2.0,5.0,71.4,0.0,2.4,0.4,0.0,0,0.0,0.0,6.0,18.0,33.3,21.0,5.0,57.1,42.0,6.0,100.0,60.2,12.0,1.0,8.3,0.0,6.5,2018-2019,-2,0,0,-2,0,2,15,Amiens_Lyon,0,0,0,1,0,0,1


In [10]:
resultat = pd.concat([a, b], sort=False)

In [12]:
resultat = affichage_colonne(resultat)

In [13]:
resultat

,DateTime,Comp,Round,Day,Venue,Team,Classement,Formation,Result,GF,GA,Opponent,Past_Matches,CumulativeWins,CumulativeDraws,CumulativeLosses,Attendance,Captain,Referee,xG,xGA,Poss,Match Report,Notes,Saison,Standard_Gls,Standard_Sh,Standard_SoT,Standard_SoT%,Standard_G/Sh,Standard_G/SoT,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_xG,Expected_npxG,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Take-Ons_Att,Take-Ons_Succ,Take-Ons_Succ%,Take-Ons_Tkld,Take-Ons_Tkld%,Carries_Carries,Carries_TotDist,Carries_PrgDist,Carries_PrgC,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Rec,Receiving_PrgR,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Challenges_Tkl,Challenges_Att,Challenges_Tkl%,Challenges_Lost,Blocks_Blocks,Blocks_Sh,Blocks_Pass,Int,Tkl+Int,Clr,Err,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,Ast,xAG,xA,KP,1/3,PPA,CrsPA,PrgP,Performance_SoTA,Performance_GA,Performance_Saves,Performance_Save%,Performance_CS,Performance_PSxG,Performance_PSxG+/-,Penalty Kicks_PKatt,Penalty Kicks_PKA,Penalty Kicks_PKsv,Penalty Kicks_PKm,Launched_Cmp,Launched_Att,Launched_Cmp%,Passes_Att (GK),Passes_Thr,Passes_Launch%,Passes_AvgLen,Goal Kicks_Att,Goal Kicks_Launch%,Goal Kicks_AvgLen,Crosses_Opp,Crosses_Stp,Crosses_Stp%,Sweeper_#OPA,Sweeper_AvgDist,GD,Points,Points_Cum,GD_Cum,GF_Cum,GA_Cum,MatchID,IsWin,IsDraw,IsLoss
218,2024-01-12 21:00:00,Ligue 1,18,Fri,Home,Marseille,NaN,NaN,NaN,NaN,NaN,Strasbourg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,2023-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
334,2024-01-12 21:00:00,Ligue 1,18,Fri,Away,Strasbourg,NaN,NaN,NaN,NaN,NaN,Marseille,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,2023-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,2024-01-13 17:00:00,Ligue 1,18,Sat,Home,Monaco,NaN,NaN,NaN,NaN,NaN,Reims,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,2023-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299,2024-01-13 17:00:00,Ligue 1,18,Sat,Away,Reims,NaN,NaN,NaN,NaN,NaN,Monaco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,2023-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
375,2024-01-13 21:00:00,Ligue 1,18,Sat,Home,Ren

In [45]:
a.dropna(subset=["Date", "Time", "Round"], inplace=True)

In [46]:
a = preprocess_intitial(a, mapping_equipe)


In [47]:
a.dropna(subset=["Date", "Time", "Round"], inplace=True)
a = preprocess_intitial(a, mapping_equipe)

aujourdhui = datetime.now()
a = a[(a['DateTime'] >= aujourdhui)]
a = a.sort_values(by='DateTime')

# Prendre la première date
premiere_date_proche = a['DateTime'].iloc[0] if not a.empty else None

# Afficher la première date proche
dix_jours = timedelta(days=10)+ premiere_date_proche
a = a[(a['DateTime'] <= dix_jours)]

In [49]:

# Prendre la première date
premiere_date_proche = a['DateTime'].iloc[0] if not a.empty else None

# Afficher la première date proche
dix_jours = timedelta(days=10)+ premiere_date_proche
a = a[(a['DateTime'] <= dix_jours)]

In [14]:

mapping_equipe = {
    'Nimes': 'Nîmes',
    'Paris S-G': 'Paris Saint Germain',
    'Saint Etienne': 'Saint-Étienne'
}

a.dropna(subset=["Date", "Time"], inplace=True)

a['DateTime'] = pd.to_datetime(a['Date'] + ' ' + a['Time'])
a = a[['DateTime'] + [col for col in a if col != 'DateTime']]

aujourdhui = datetime.now()


# Garder que les matchs 'Ligue 1' (pas de matchs de Coupe)
a = a[a["Comp"] == "Ligue 1"]


# Filtrage par date
a = a[(a['DateTime'] >= aujourdhui)]
#& (a['DateTime'] <= dans_dix_jours)


# Trier le DataFrame filtré par date
a = a.sort_values(by='DateTime')

# Prendre la première date
premiere_date_proche = a['DateTime'].iloc[0] if not a.empty else None

# Afficher la première date proche
dix_jours = timedelta(days=10)+ premiere_date_proche

a['Team'] = a['Team'].map(mapping_equipe).fillna(a['Team'])


# Extraire uniquement le numéro de chaque journée
a['Round'] = a['Round'].str.extract(r'(\d+)').astype(int)

a = a[(a['DateTime'] <= dix_jours)]


In [14]:
a

,DateTime,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,...,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team
218,2024-01-12 21:00:00,2024-01-12,21:00,Ligue 1,18,Fri,Home,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Marseille
334,2024-01-12 21:00:00,2024-01-12,21:00,Ligue 1,18,Fri,Away,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Strasbourg
97,2024-01-13 17:00:00,2024-01-13,17:00,Ligue 1,18,Sat,Home,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Monaco
299,2024-01-13 17:00:00,2024-01-13,17:00,Ligue 1,18,Sat,Away,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Reims
375,2024-01-13 21:00:00,2024-01-13,21:00,Ligue 1,18,Sat,Home,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Rennes
62,2024-01-13 21:00:00,2024-01-13,21:00,Ligue 1,18,Sat,Away,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Nice
631,2024-01-14 13:00:00,2024-01-14,13:00,Ligue 1,18,Sun,Away,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Lorient
175,2024-01-14 13:00:00,2024-01-14,13:00,Ligue 1,18,Sun,Home,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Lille
412,2024-01-14 15:00:00,2024-01-14,15:00,Ligue 1,18,Sun,Away,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Montpellier
552,2024-01-14 15:00:00,2024-01-14,15:00,Ligue 1,18,Sun,Home,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Metz


In [41]:
c = pd.read_csv("SOCCER_201223_18h-2.csv", index_col=0)

In [46]:
add_new_matches(c,b)

KeyError: 'DateTime'

In [49]:
A['DateTime'] = pd.to_datetime(A['Date'] + ' ' + A['Time'])
A.drop(["Date", "Time"], axis=1, inplace=True)

In [50]:
A['DateTime'] = pd.to_datetime(A['Date'] + ' ' + A['Time'])
A = A[['DateTime'] + [col for col in A if col != 'DateTime']]
aujourdhui = datetime.now()

A = A[(A['DateTime'] >= aujourdhui) & (A["Round"] == 18)]


A['Team'] = A['Team'].map(mapping_equipe).fillna(A['Team'])

# Garder que les matchs 'Ligue 1' (pas de matchs de Coupe)
A = A[A["Comp"] == "Ligue 1"]

# Extraire uniquement le numéro de chaque journée
A['Round'] = A['Round'].str.extract(r'(\d+)').astype(int)



In [51]:

mapping_equipe = {
    'Nimes': 'Nîmes',
    'Paris S-G': 'Paris Saint Germain',
    'Saint Etienne': 'Saint-Étienne'
}

test = a[1].copy()


In [ ]:
preprocess(test, mapping_equipe)

In [17]:
eeee

In [52]:

A['Team'] = A['Team'].map(mapping_equipe).fillna(A['Team'])

# Garder que les matchs 'Ligue 1' (pas de matchs de Coupe)
A = A[A["Comp"] == "Ligue 1"]

# Extraire uniquement le numéro de chaque journée
A['Round'] = A['Round'].str.extract(r'(\d+)').astype(int)



In [53]:
A.sort_values(by="DateTime")

,DateTime,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team
149,2023-08-11 21:00:00,Ligue 1,1,Fri,Away,D,1.0,1.0,Nice,1.3,1.2,60.0,29436.0,Benjamin André,4-2-3-1,Benoît Bastien,Match Report,NaN,Lille
44,2023-08-11 21:00:00,Ligue 1,1,Fri,Home,D,1.0,1.0,Lille,1.2,1.3,40.0,29436.0,Dante,4-3-3,Benoît Bastien,Match Report,NaN,Nice
193,2023-08-12 17:00:00,Ligue 1,1,Sat,Home,W,2,1,Reims,1.0,0.8,42.0,63787.0,Valentin Rongier,4-4-2,Eric Wattellier,Match Report,NaN,Marseille
281,2023-08-12 17:00:00,Ligue 1,1,Sat,Away,L,1.0,2.0,Marseille,0.8,1.0,58.0,63787.0,Yunis Abdelhamid,4-2-3-1,Eric Wattellier,Match Report,NaN,Reims
0,2023-08-12 21:00:00,Ligue 1,1,Sat,Home,D,0.0,0.0,Lorient,1.2,0.1,78.0,47000.0,Danilo Pereira,4-3-3,Bastien Dechepy,Match Report,NaN,Paris Saint Germain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676,NaT,Ligue 1,28,Sun,Away,NaN,NaN,NaN,Paris S-G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Clermont Foot
677,NaT,Ligue 1,29,Sun,Home,NaN,NaN,NaN,Montpellier,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Clermont Foot
678,NaT,Ligue 1,30,Sun,Away,NaN,NaN,NaN,Lens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Clermont Foot
679,NaT,Ligue 1,31,Sun,Home,NaN,NaN,NaN,Reims,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Clermont Foot


In [54]:
aujourdhui = datetime.now()

A = A[(A['DateTime'] >= aujourdhui) & (A["Round"] == 18)]

In [56]:
A.to_csv("A.csv")

In [59]:
a = scrape_latest_ligue1_data()

/home/onyxia/work/Fbref_model/Projet-python/Fbref_alex/scrapping.py:88: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  team_data = pd.read_html(team_response.text, match="Scores")[0]
/home/onyxia/work/Fbref_model/Projet-python/Fbref_alex/scrapping.py:153: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/home/onyxia/work/Fbref_model/Projet-python/Fbref_alex/scrapping.py:153: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/home/onyxia/work/Fbref_model/Projet-python/Fbref_alex/scrapping.py:153: FutureWarning: Passing l

In [68]:
D = add_new_matches(base_initiale=b, base_nouvelle=a)

In [60]:
b = pd.read_csv("/home/onyxia/work/Fbref_model/Projet-python/Fbref_alex/SOCCER_201223_18h.csv")